In [ ]:
from os import listdir
from os.path import isfile, join
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !rm -r citation_sentences
!rm -r output

## SciBERT+CRF

In [ ]:
!pip install -U transformers
!pip install pytorch-crf

In [ ]:

#sample prediction
from io import open
import unicodedata
import string
import re
import random
import pickle 
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils import data 
from transformers import BertTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import os
from transformers import BertPreTrainedModel, BertModel
from torchcrf import CRF
import timeit
import subprocess
from tqdm import tqdm, trange
from transformers import AdamW, get_linear_schedule_with_warmup
from matplotlib import pyplot as plt 
import datetime
# from config import Config as config
import spacy
import torch
tokenizer = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner"])
log_soft = F.log_softmax
import sys
from optparse import OptionParser
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [ ]:
from nltk import pos_tag
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [ ]:
def remove_colon_and_period(sentence):
    # punc = !.:
    return sentence.translate(str.maketrans('', '', ".:"))
    
def preprocess_corpus(sentence):
  # text = text.lower()
  # text = re.compile('[-]').sub('',text)
  # regex = re.compile('[^a-zA-Z]')
  #First parameter is the replacement, second parameter is your input string
  # text = regex.sub(' ', text)
  sentence = remove_colon_and_period(sentence)
  words = []
  for word,pos in pos_tag([w for w in word_tokenize(sentence)]):
      if '-' in word and word.index('-')+1 < len(word) and word[word.index('-')+1].isupper():
        word = word.replace('-','')
      words.append(word)
  return ' '.join(words)

class Bert_CRF(BertPreTrainedModel):
    def __init__(self, config):
        super(Bert_CRF, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)
        self.init_weights()
        self.crf = CRF(self.num_labels, batch_first=True)    
    
    def forward(self, input_ids, attn_masks, labels=None):  # dont confuse this with _forward_alg above.
        outputs = self.bert(input_ids, attn_masks)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        emission = self.classifier(sequence_output)        
        attn_masks = attn_masks.type(torch.uint8)
        if labels is not None:
            loss = -self.crf(log_soft(emission, 2), labels, mask=attn_masks, reduction='mean')
            return loss
        else:
            prediction = self.crf.decode(emission, mask=attn_masks)
            return prediction

def get_algo_entities(sentence,scibertmodel):
    spacy_tokenizer = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner"])
    # sentence = word+" "+ "is a very popular classifier due to its robustness against large feature vectors and sparse data."
#     sentence = '''However, these word-granularity models
# are unable to fully capture the semantic features embedded in sentences, some-
# times even produce noise and thus hurt the performance of sentence matching.
# Eventually, more and more researchers turn to design semantic matching strategy
# combing word and phrase granularity, such as MultiGranCNN [24], MV-LSTM
# [15], MPCM [22], BiMPM [21], DIIN [3].'''
    sentence = preprocess_corpus(sentence)
    tokens = spacy_tokenizer(sentence) 
    bert_tokens = np.array(['[CLS]'])
    orig_to_tok_map = np.array([], dtype="i")
    begins = np.array([], dtype="i")
    ends = np.array([], dtype="i")
    original_token = np.array([])
    bert_tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    offset = 0
    for tmp_tok in tokens:
        new_token = np.array(bert_tokenizer.tokenize(tmp_tok.text))
        original_token = np.append(original_token, new_token)
        orig_to_tok_map = np.append(orig_to_tok_map, bert_tokens.shape[0])
        bert_tokens = np.append(bert_tokens, new_token)
        offset = sentence.find(tmp_tok.text)
        begins = np.append(begins, offset)
        ends = np.append(ends, offset + len(tmp_tok.text))
        offset += len(tmp_tok.text)
    bert_tokens = np.append(bert_tokens, '[SEP]')
    tok_ids = bert_tokenizer.convert_tokens_to_ids(bert_tokens)
    attn_mask = [1] * len(tok_ids)   
    tok_ids = torch.LongTensor([tok_ids])   
    attn_mask = torch.LongTensor([attn_mask])
    # token_ids, attn_mask, org_tok_map
    inputs = {'input_ids': tok_ids.to(device),
              'attn_masks' : attn_mask.to(device)
            }  
    with torch.torch.no_grad():
        tag_seqs = scibertmodel(**inputs)
    # for tok,tag in zip(bert_tokens,tag_seqs[0]):
    #  print(tok,tag) 
    # print(bert_tokens)
    pretok_sent = ""
    prelabel = ""
    # Removing the first and last tokens
    bert_tokens, tag_seqs[0] = bert_tokens[1:-1],tag_seqs[0][1:-1]
    for tok,tag in zip(bert_tokens,tag_seqs[0]):
      if tok.startswith("##"):
          pretok_sent += tok[2:]
          prelabel +=str(tag)
      else:
          pretok_sent += " " + tok
          prelabel += " " + str(tag)
    
    original_tokens = [tok for tok in word_tokenize(pretok_sent)]
    predicted_labels = [tok for tok in word_tokenize(prelabel)]

    predicted_entities = []
    for tok,tag in zip(original_tokens,predicted_labels):
      # print(tok,tag)
      if (str(tag2idx['B-Algo']) in tag or str(tag2idx['I-Algo']) in tag):
        if len(tok)>1 and tok.isdigit()==False:
          predicted_entities.append(tok)
    # print(sentence)
    # print(predicted_entities)
    return predicted_entities

In [ ]:
bert_model = 'allenai/scibert_scivocab_uncased'
with open('/content/drive/MyDrive/models/model_algo/tag2idx.pkl','rb') as f:
  tag2idx = pickle.load(f)
# checkpt = torch.load('/content/drive/MyDrive/models/model_algo/model_0.pt',map_location=torch.device('cpu'))
checkpt = torch.load('/content/drive/MyDrive/models/model_scibert_paperswithcode/model/model_0.pt',map_location=torch.device('cpu'))
scibertmodel = Bert_CRF.from_pretrained(bert_model, num_labels = len(tag2idx))
scibertmodel.to(device)
scibertmodel.load_state_dict(checkpt['model_state_dict'],strict = False)
get_algo_entities('implement strong baselines with features available in FiCLS, including GLU (Hadiwinoto et al., 2019), GlossBERT (Huang et al., 2019) and BEM (Blevins and Zettlemoyer, 2020), and use the same settings as our model for a fair comparison.',scibertmodel)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing Bert_CRF: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing Bert_CRF from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Bert_CRF from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Bert_CRF were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


['baselines', 'ficls', 'glossbert', 'bem', 'model']

## Keyphrase Extraction

In [ ]:
!python -m nltk.downloader stopwords
!python -m nltk.downloader universal_tagset
!python -m spacy download en # download the english model
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
import pke
import string
def get_keyphrases(text = "abcd efg hijk I am this"):
  extractor = pke.unsupervised.MultipartiteRank()
  extractor.load_document(input=text)
  pos = {'NOUN', 'PROPN', 'ADJ'}
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.candidate_selection(pos=pos, stoplist=stoplist)
  extractor.candidate_weighting(alpha=1.1,
                                threshold=0.74,
                                method='average')
  keyphrases = extractor.get_n_best(n=1000)
  return keyphrases

In [ ]:
import pke
def get_keyphrases(text = "abcd efg hijk I am this"):
  # TextRank

  # define the set of valid Part-of-Speeches
  pos = {'NOUN', 'PROPN', 'ADJ'}

  # 1. create a TextRank extractor.
  extractor = pke.unsupervised.TextRank()

  # 2. load the content of the document.
  extractor.load_document(input=text
                          )

  # 3. build the graph representation of the document and rank the words.
  #    Keyphrase candidates are composed from the 33-percent
  #    highest-ranked words.
  extractor.candidate_weighting(window=2,
                                pos=pos,
                                top_percent = 0.8
                                )

  # 4. get the 10-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=1000)
  return keyphrases
########################################################### Single Rank #####################
# import pke
# def get_keyphrases(text = "abcd efg hijk I am this"):
#   # define the set of valid Part-of-Speeches
#   pos = {'NOUN', 'PROPN', 'ADJ'}

#   # 1. create a SingleRank extractor.
#   extractor = pke.unsupervised.SingleRank()

#   # 2. load the content of the document.
#   extractor.load_document(input=text,
#                           language='en',
#                           normalization=None)

#   # 3. select the longest sequences of nouns and adjectives as candidates.
#   extractor.candidate_selection(pos=pos)

#   # 4. weight the candidates using the sum of their word's scores that are
#   #    computed using random walk. In the graph, nodes are words of
#   #    certain part-of-speech (nouns and adjectives) that are connected if
#   #    they occur in a window of 10 words.
#   extractor.candidate_weighting(window=10,
#                                 pos=pos)

#   # 5. get the 10-highest scored candidates as keyphrases
#   keyphrases = extractor.get_n_best(n=1000)
#   return keyphrases

# ################################ TopicRank ##############################
# import pke
# import string
# from nltk.corpus import stopwords
# def get_keyphrases(text = "abcd efg hijk I am this"):
#   # 1. create a TopicRank extractor.
#   extractor = pke.unsupervised.TopicRank()

#   # 2. load the content of the document.
#   extractor.load_document(input=text)

#   # 3. select the longest sequences of nouns and adjectives, that do
#   #    not contain punctuation marks or stopwords as candidates.
#   pos = {'NOUN', 'PROPN', 'ADJ'}
#   stoplist = list(string.punctuation)
#   stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
#   stoplist += stopwords.words('english')
#   extractor.candidate_selection(pos=pos, stoplist=stoplist)

#   # 4. build topics by grouping candidates with HAC (average linkage,
#   #    threshold of 1/4 of shared stems). Weight the topics using random
#   #    walk, and select the first occuring candidate from each topic.
#   extractor.candidate_weighting(threshold=0.74, method='average')

#   # 5. get the 10-highest scored candidates as keyphrases
#   keyphrases = extractor.get_n_best(n=1000)
#   return keyphrases

# ########################## PositionRank ##################################
# import pke

# def get_keyphrases(text = "abcd efg hijk I am this"):
#   # define the valid Part-of-Speeches to occur in the graph
#   pos = {'NOUN', 'PROPN', 'ADJ'}

#   # define the grammar for selecting the keyphrase candidates
#   grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

#   # 1. create a PositionRank extractor.
#   extractor = pke.unsupervised.PositionRank()

#   # 2. load the content of the document.
#   extractor.load_document(input=text,
#                           language='en',
#                           normalization=None)

#   # 3. select the noun phrases up to 3 words as keyphrase candidates.
#   extractor.candidate_selection(grammar=grammar,
#                                 maximum_word_number=3)

#   # 4. weight the candidates using the sum of their word's scores that are
#   #    computed using random walk biaised with the position of the words
#   #    in the document. In the graph, nodes are words (nouns and
#   #    adjectives only) that are connected if they occur in a window of
#   #    10 words.
#   extractor.candidate_weighting(window=10,
#                                 pos=pos)

#   # 5. get the 10-highest scored candidates as keyphrases
#   keyphrases = extractor.get_n_best(n=1000)
#   return keyphrases

get_keyphrases('''In this paper, we propose a novel embedding
model, named ConvKB, for knowledge base
completion. Our model ConvKB advances
state-of-the-art models by employing a convolutional neural network, so that it can capture
global relationships and transitional characteristics between entities and relations in knowledge bases. In ConvKB, each triple (head entity, relation, tail entity) is represented as a 3-
column matrix where each column vector represents a triple element. This 3-column matrix
is then fed to a convolution layer where multiple filters are operated on the matrix to generate different feature maps. These feature maps
are then concatenated into a single feature vector representing the input triple. The feature
vector is multiplied with a weight vector via
a dot product to return a score. This score
is then used to predict whether the triple is
valid or not. Experiments show that ConvKB
achieves better link prediction performance
than previous state-of-the-art embedding models on two benchmark datasets WN18RR and
FB15k-237.'''
)

[('better link prediction performance', 0.08447905672650476),
 ('different feature maps', 0.07769642479523654),
 ('art embedding mod\x02els', 0.06545816355841445),
 ('feature maps', 0.06374581673878935),
 ('benchmark datasets wn18rr', 0.06335991254487858),
 ('convo\x02lutional neural network', 0.06335834254487857),
 ('column vector', 0.05483697997050867),
 ('art models', 0.05121468114044349),
 ('feature', 0.049795248682342155),
 ('input triple', 0.04701905911337171),
 ('triple element', 0.04701863911337171),
 ('knowl\x02edge bases', 0.047018329113371705),
 ('knowledge base', 0.047017919113371706),
 ('previous state', 0.04224046836325237),
 ('dot product', 0.04224010836325237),
 ('weight vector', 0.04224005836325238),
 ('multi\x02ple filters', 0.04223967836325237),
 ('convolution layer', 0.04223964836325237),
 ('column matrix', 0.04223944836325238),
 ('tail entity', 0.04223935836325237),
 ('head en\x02tity', 0.04223930836325237),
 ('transitional character\x02istics', 0.04223914836325237

## Execute Pipeline

In [ ]:
def execute_pipeline(citation_sentences,scibertmodel):
    ##Getting named entities from citation sentences (sequence labeller)
    ents = set()
    for s in citation_sentences:
        entity = get_algo_entities(s,scibertmodel)
        ents.update(entity)

    temp_ents = list(ents)
    for word in ents:
      if word[-1] == 's' and word[:-1] in ents:    #Removing plural duplicates of words 
        temp_ents.remove(word)
    ents = temp_ents
    predicted_algos = ents
    ########## Keyphrase ##############################
    words = []
    for sentence in citation_sentences:
      for word,pos in pos_tag([w for w in word_tokenize(sentence) if w not in stop]):
          if pos == 'NNP':
            if '-' in word and word[word.index('-')+1].isupper():
              word = word.replace('-','')
          words.append(word)
    citationtext = " ".join(words)
    keyphrases = get_keyphrases(citationtext)
    keyphrases = [k.replace('-','') for k,s in keyphrases[:len(keyphrases)] if len(k)<=15 and len(word_tokenize(k))<=2]
    # print(keyphrases)
    # for word in keyphrases:
    #   if word[-1] == 's' and word[:-1] in ents:    #Removing plural duplicates of words 
    #     keyphrases.remove(word)
    
    ####################################################
    predicted_algos = [ent for ent in keyphrases if ent in predicted_algos]
    return keyphrases,predicted_algos
    # return predicted_algos

## Extracting Entities

In [ ]:
!unzip citation_sentences.zip

In [ ]:
!mkdir 'output'
%cd output
!mkdir 'keyphrase'
!mkdir 'final output'
%cd ../

/content/output
/content


In [ ]:
parentdir = '.'
citdir = "citation_sentences"
outputdir = 'output'
inputfolder = join(parentdir, citdir)
outputfolder = join(parentdir, outputdir)

fileslist = [f for f in listdir(inputfolder) if isfile(
        join(inputfolder, f))]
fileslist.sort()

for i,filename in tqdm(enumerate(fileslist), total=len(fileslist)):
  print(filename)
  citation_sentences =  []
  with open(inputfolder+'/'+filename) as f:
    for line in f:
      if line[0] == '[':
        continue
      citation_sentences.append(line.strip("\n"))
  keyphrases, final_predicted = execute_pipeline(citation_sentences,scibertmodel)
  with open(outputfolder+"/" + "keyphrase" +"/"+filename, 'w',encoding = 'utf-8') as f:
    for ent in keyphrases:
      f.write(ent+"\n")
  with open(outputfolder+"/" + "final output" +"/"+filename, 'w',encoding = 'utf-8') as f:
    for ent in final_predicted:
      f.write(ent+"\n")

  0%|          | 0/84 [00:00<?, ?it/s]

1909.00426.txt


  1%|          | 1/84 [00:20<28:24, 20.53s/it]

1909.08402v1.txt


  2%|▏         | 2/84 [00:37<25:01, 18.31s/it]

...

 98%|█████████▊| 82/84 [32:30<00:50, 25.48s/it]

p1877.txt


 99%|█████████▉| 83/84 [32:31<00:18, 18.29s/it]

you_can_teach_an_old_dog_new_t.txt


100%|██████████| 84/84 [33:04<00:00, 23.63s/it]


In [ ]:
final_predicted

(['word entity', 'et al', 'mentions entity'], [])

In [ ]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/keyphrase/ (stored 0%)
  adding: output/keyphrase/1911.09419v2.txt (deflated 18%)
  adding: output/keyphrase/CLINE.txt (deflated 46%)
  adding: output/keyphrase/2021.eacl-demos.4.txt (deflated 41%)
  adding: output/keyphrase/2109.05782.txt (deflated 32%)
  adding: output/keyphrase/2021.findings-emnlp.50.txt (deflated 34%)
  adding: output/keyphrase/2021.findings-emnlp.63.txt (deflated 40%)
  adding: output/keyphrase/2021.findings-emnlp.23.txt (deflated 34%)
  adding: output/keyphrase/2021.findings-emnlp.54.txt (deflated 40%)
  adding: output/keyphrase/TransG.txt (deflated 24%)
  adding: output/keyphrase/LoPAD.txt (deflated 39%)
  adding: output/keyphrase/2021.naacl-main.41.txt (stored 0%)
  adding: output/keyphrase/Mask-Guided Region Attention Network.txt (deflated 32%)
  adding: output/keyphrase/DOLORES Deep Contextualized Knowledge Graph Embeddings.txt (deflated 44%)
  adding: output/keyphrase/SLGAT-Soft Labels Guided Graph.txt (deflated

In [ ]:
annotations = {
              'JARKA.txt' : ['JAPE','MultiKE','AttrE','MuGNN','BootEA','GCNs','KDCoE'],
              'BiWalkLDA.txt':['LDAP', 'LRLSLDA', 'SIMCLDA','KATZHMDA','IRWRLDA','RWRlncD','MFLDA'],
              'TemporalGAT.txt':['G-SAGE','GAT', 'Node2Vec', 'GraphSAGE', 'GCN-AE','GAT-AE','Know-Evolve', 'DynamicTriad','DynGEM', 'DySAT','DANE'],
              'LoPAD.txt':['ALSO','COMBN','MBOM', 'iForest','LOF'],
              'DELAFO.txt' : ['ResNet','ARMA','ARIMA','LSTM','GRU','SA','AA'],

'A Benchmarking Study of Embedding based entity alignment for Knowledge Graphs (Springer).txt' : ['MTransE','IPTransE', 'JAPE', 'KDCoE', 'BootEA','GCNAlign','AttrE','IMUSE','SEA','RSN4EA','MultiKE','RDGCN','TransH', 'TransR', 'TransD', 'ProjE', 'ConvE', 'HolE', 'SimplE','RotatE','OpenKE','TuckER',
'OpenEA','LogMap','NTAM','AC2Vec','Label2Vec','DistMult','Complex', 'KBGAN','DSKG','OTEA','NAEA','TransE','RGCN','AKE'],

'A Proximity Weighted Evidential k.txt' : ['DTs','PEkNN','kNN', 'C4.5', 'NB','SMOTE', 'EKNN', 'WKNN', 'CCWKNN', 'kENN', 'GMDKNN','CCPDT', 'HDDT','iHDwDT'],


'An Improved Proportionate Normalized.txt' : ['NLMS','PNLMS','IPNLMS','MPNLMS'],

'AutoSUM - Automating Feature Extraction.txt' : ['LDA','LSTM-CRF','RELIN','SUMMARUM','FACES','ES-LDAext','ESA','DIVERSUM','CD','LinkSUM','MPSUM'],

'Bottom-Up and Top-Down Graph Pooling.txt' : ['SAGPool','GCN','Graph-SAGE','SortPool','Set2Set','SAGPool','gPool','DiﬀPool'],


'CACRNN- A Context-Aware.txt' : ['ARIMA','HA','DCRNN','STGCN','LSTM'],	# Short form issue. Check what results you are getting for algo output

'Chinese Sentence Semantic Matching.txt' : ['CNN','BiLSTM', 'BiMPM','DFF','DeepMatchtree','ARC-II','Match-Pyramid','Match-SRNN','MultiGranCNN', 'MV-LSTM', 'MPCM', 'DIIN','MIX','WMD',
'CBOW'],

'Correlation-aware Deep Generative Model for.txt' : ['OC-SVM','IF','DSEBM','DAGMM','AnoGAN','ALAD'],

'CrowdQM- Learning Aspect-Level User.txt' : ['MBoA','CRH','TrustAnswer','CATD','CrowdQM-no-aspect'],

'Deep Cost-Sensitive Kernel Machine for.txt' : ['BRNN-C','BRNN-D','Para2Vec','VDiscover','VulDeePecker','BRNN-SVM','Att-BGRU','Text CNN','MDSAE','OC-DeepSVDD'],

'GAMMA-A Graph and Multi-view.txt' :['NeuACF','HERec','NeuMF','MF','BPR','FMG','KGAT','SemRec', 'MCRec', 'HeteRec'],

'Joint Relational Dependency Learning.txt' : ['RNN','LSTM','BPR-MF','TranRec','GRU4Rec','FPMC','MARank'],

'Mask-Guided Region Attention Network.txt' : ['PDC','AACN','PAN','PAR','DPFL','DaF','NFST','OIM','SVDNet','Spindle-Net','DLPAR','MSCAN'],



'Multiple Demographic Attributes.txt' : ['POP','JNE','SNE','ETN','ETNA'],		# Short form issue


'Passenger demand forecasting with multi-task convolutional recurrent neural networks (accepted version).txt' : ['HA','ARIMA','SARIMA','OLSR','MLP','LSTM','XGBoost','DMVST-Net'],

'Relation Embedding for Personalised.txt' : ['GeoSAGE', 'GraphSAGE','PMF','GeoMF','Rank-GeoFM','GeoSoCa','ST-LDA','TransRec','STA','GCN','DeepWalk','Node2Vec','CKN','MF','KGE'],

'Robust Attribute and Structure.txt' : ['VGAE','Graph2Gauss','LINE','DeepWalk','node2vec','DVNE','TADW','GraphSAGE','Graph2Gauss','DANE','GLACE','G2G'],

'SAFE.txt' : ['LIWC','VGG-19','att-RNN','TI-CNN','EANN'],

'SGCN-A Graph Sparsifier.txt' : ['GCN','DUIF','RP','SS','DeepWalk','GraphSAGE'],		# short form

'SLGAT-Soft Labels Guided Graph.txt' : ['MLP','ManiReg','SemiEmb','LP','DeepWalk','ICA','Planetoid','ChebyNet','GCN','MoNet','GAT','SPAGAN','GraphHeat','GWNN','LINE'],

'SoRecGAT- Leveraging Graph Attention.txt' : ['SAMN','DeepSoR','SBPR','TrustSVD','NeuMF','GMF','BPR','MF','RecGAT','SocialMF','SoReg','SR-GNN','GraphRec','PinSage','ATRank'],

'SubRank-Subgraph Embeddings.txt' : ['DeepWalk','node2vec','LINE','VERSE','VerseAvg ','sub2vec','DeepCas','DeepHawkes','ParagraphVector'],


'Towards Understanding Transfer Learning.txt': ['RPROJ3', 'PCA','TCA', 'GFK' , 'SA' , 'KMM' ,'MSDA', 'GFK','CORAL','ITL','GTL','LSDT','DAN' ,'RevGrad','ADDA' ,'DDC','DAN'],

'CLINE.txt': ['BERT','RoBERTa','TextFooler','FreeLB','GPT'],

'DOLORES Deep Contextualized Knowledge Graph Embeddings.txt' : ['RESCAL','TRANSE','DISTMULT','TRANSD','COMPLEX','CONVE','MANIFOLDE','HOLE','CONVKB','KBGAN','NODE2VEC','PTransE','PRA','RNN-Path','RNN-Path-entity'],

'KBGAN.txt': ['RESCAL','TRANSE','DISTMULT','COMPLEX','TRANSH','TRANSR','TRANSD','MANIFOLDE','HOLE','CONVE','SEQGAN','IRGAN','WGAN','GOGAN','GANs',' Fast-TransX'],

'new.txt' : ['XGBoost','TabNet','NODE','DNF-Net','1D-CNN','SVM','CatBoost'],

'SCIBERT.txt' : ['BERT','ELMo','GPT','BIOBERT','CLINICALBERT']

               }


In [ ]:
for k,v in annotations.items():
  with open("annotations"+"/"+k, 'w', encoding = 'utf-8') as f:
    for ent in v:
      f.write(ent+"\n")

In [ ]:
!zip -r annotations.zip annotations

  adding: annotations/ (stored 0%)
  adding: annotations/CLINE.txt (stored 0%)
  adding: annotations/Chinese Sentence Semantic Matching.txt (deflated 14%)
  adding: annotations/BiWalkLDA.txt (deflated 15%)
  adding: annotations/JARKA.txt (deflated 2%)
  adding: annotations/TemporalGAT.txt (deflated 14%)
  adding: annotations/SubRank-Subgraph Embeddings.txt (deflated 9%)
  adding: annotations/DOLORES Deep Contextualized Knowledge Graph Embeddings.txt (deflated 16%)
  adding: annotations/Bottom-Up and Top-Down Graph Pooling.txt (deflated 22%)
  adding: annotations/Deep Cost-Sensitive Kernel Machine for.txt (deflated 7%)
  adding: annotations/Robust Attribute and Structure.txt (deflated 21%)
  adding: annotations/DELAFO.txt (deflated 6%)
  adding: annotations/SCIBERT.txt (deflated 11%)
  adding: annotations/A Benchmarking Study of Embedding based entity alignment for Knowledge Graphs (Springer).txt (deflated 29%)
  adding: annotations/AutoSUM - Automating Feature Extraction.txt (deflated 